In [40]:
%pip install pyttsx3
%pip install openai==0.28
%pip install SpeechRecognition 
%pip install scikit-learn





Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: scikit-learn in c:\users\marid\appdata\local\programs\python\python311\lib\site-packages (1.5.2)




[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Assunto: Siamo un'azienda produttrice di giocattoli, attraverso un form online i clienti possono realizzare il loro gioco personalizzandolo completamente che verrà successivamente inviato alla linea di assemblaggio che si occuperà di realizzare il prodotto ed effettuare un controllo qualità.

I PROCESSI DI NOSTRO INTERESSE SONO: L'ASSEMBLAGGIO E IL CONTROLLO QUALITA'

PROCESSO AS IS:
- Ricezione dell'ordine:
L'operatore riceve le foto del prodotto finito e la distinta base.
- Istruzioni:
L'operatore usa la foto per capire come assemblare il prodotto e la BOM per identificare i pezzi, senza istruzioni specifiche.
- Assemblaggio:
Procede con l'assemblaggio basandosi sulla foto e l'elenco dei pezzi deducendo la sequenza corretta.
- Controllo qualità (compito affidato ad un altro operatore in un'altra stazione):
Confronta visivamente il prodotto finito con la foto per verificarne la correttezza.


Assunto : La distinta base (BOM) e la foto del prodotto finito vengono automaticamente tradotte in un manuale di istruzioni dettagliato tramite AI.
PROCESSO TO BE:
- Ricezione dell'ordine:
riceve la commessa con il nome del prodotto finito e interrgoa il chatbot per ricevere assistenza.
Il chatbot riceve il manuale generato  che viene automaticamente salvato e guida l'operatore passo-passo nel montaggio, fornendo istruzioni chiare sui pezzi da utilizzare e l'ordine di assemblaggio.

- Assemblaggio:
L'operatore segue le istruzioni del chatbot, che fornisce supporto e feedback in tempo reale durante l'assemblaggio.

- Controllo qualità:
Il pezzo scorre e attraverso object detection viene eseguita una scansione visiva del prodotto finito, verificando automaticamente la correttezza del montaggio e segnalando eventuali errori.


Criticità del Processo As Is:
La mancanza di un manuale dettagliato obbliga l'operatore a dedurre la sequenza di montaggio, con il rischio di montare i pezzi nell'ordine sbagliato.
Controllo qualità soggettivo:
Il controllo qualità è manuale e visivo, basato sull'osservazione dell'operatore, il che può portare a errori non rilevati o difetti nascosti.
Non ci sono strumenti che forniscano un feedback immediato all'operatore durante l'assemblaggio, quindi eventuali errori vengono scoperti solo alla fine, aumentando i costi di correzione.

Vantaggi implementazione chatbot:
meno errori grazie alle istruzioni, un operatore in meno e tempo risparmiato



In [39]:
#prompt3 con meno chunk_ pare funzioni!!! (SI) 
import re
import openai
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Imposta direttamente la chiave API qui
openai.api_key = 'sk-du21AXQeS5dbEhauUmmaLCRmm9jzyYKY7wemdgGIFET3BlbkFJ4r1IqdlWQ_lhECmi3PGJCy-V0M-_z_7EqiTT7jCHsA'  # Sostituisci con la tua chiave API

# Definisci i percorsi dei manuali
manuali_prodotti = {
    'giraffa': r'C:\Users\marid\OneDrive\Desktop\PSF\Giraffa_.txt',
    'treno_arancio': r'C:\Users\marid\OneDrive\Desktop\PSF\TrenoArancio_.txt',
    'treno_viola': r'C:\Users\marid\OneDrive\Desktop\PSF\TrenoViola_.txt',
}

# Funzione per caricare e dividere un manuale in chunk
def carica_e_dividi_manuali(filename, chunk_size=500):
    manual_chunks = []
    with open(filename, 'r', encoding='utf-8') as file:
        testo = file.read()
        paragrafi = testo.split("\n\n")
        chunk = ""
        for paragrafo in paragrafi:
            if len(chunk) + len(paragrafo) < chunk_size:
                chunk += paragrafo + "\n\n"
            else:
                manual_chunks.append(chunk.strip())
                chunk = paragrafo + "\n\n"
        if chunk:
            manual_chunks.append(chunk.strip())
    return manual_chunks

# Funzione per trovare il chunk rilevante con +-2 chunk
def trova_chunk_rilevante(domanda, manual_chunks, vicinanza=2):
    documenti = [domanda] + manual_chunks
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documenti)
    similarita = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    
    # Trova l'indice del chunk più rilevante
    indice_miglior_chunk = np.argmax(similarita)
    
    # Crea una lista dei chunk intorno a quello rilevante, includendo i chunk vicini
    chunk_rilevanti = []
    inizio = max(0, indice_miglior_chunk - vicinanza)
    fine = min(len(manual_chunks), indice_miglior_chunk + vicinanza + 1)
    
    # Aggiungi i chunk dalla lista ristretta
    for i in range(inizio, fine):
        chunk_rilevanti.append(manual_chunks[i])
    
    return "\n\n".join(chunk_rilevanti)  # Restituisce i chunk concatenati come un unico testo

# Funzione per interagire con OpenAI
def interroga_openai(domanda, chunk_rilevante, memoria, temperatura=0.0):
    prompt_memoria = "\n".join(memoria)
    
    prompt = f"""
    Manuale di riferimento: \n{chunk_rilevante}\n\nConversazione precedente: {prompt_memoria}\n\nDomanda: {domanda}\n
    Sei un assistente virtuale specializzato nell'assemblaggio di prodotti. Si tratta di un prodotto modulare. 
    Il tuo compito è aiutare un operatore a seguire correttamente le istruzioni di un manuale di assemblaggio.
    - Se l'utente chiede i materiali, fornisci solo la lista dei materiali necessari senza aggiungere istruzioni di assemblaggio.
    - Se l'utente chiede i moduli, fornisci solo la lista dei moduli necessari senza aggiungere istruzioni di assemblaggio.
    - Se l'utente utilizza i numeri ordinali al posto dei numeri cardinali, fornisci la fase giusta corrispondemte.
    - Se l'utente chiede istruzioni per una fase specifica, fornisci solo le istruzioni per quella fase.
    - Se l'utente chiede di procedere con la fase successiva o precedente, fornisci le istruzioni per la fase successiva o precedente.
    - Non fornire informazioni non richieste e non procedere con le fasi di assemblaggio a meno che l'utente non lo chieda esplicitamente.
    - Restituiscimi le risposte a tutte le domande.
    - Nel momento in cui arrivi alla fase finale, restituisci un messaggio in cui spieghi che il lavoro è terminato.
    """
 
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "Sei un assistente esperto che aiuta con l'assemblaggio di prodotti seguendo il manuale, non inventi nulla."},
                  {"role": "user", "content": prompt}],
        max_tokens=500,
        temperature=temperatura
    )
    return response['choices'][0]['message']['content']

# Funzione principale per generare una risposta con +-2 chunk
def genera_risposta(domanda, manual_chunks, memoria, temperatura=0.0):
    chunk_rilevante = trova_chunk_rilevante(domanda, manual_chunks)
    risposta = interroga_openai(domanda, chunk_rilevante, memoria, temperatura)
    return risposta

# Funzione per selezionare il manuale in base all'input dell'utente
def seleziona_manuale(user_input):
    if re.search(r'giraffa', user_input, re.IGNORECASE):
        return 'giraffa'
    elif re.search(r'treno arancio', user_input, re.IGNORECASE):
        return 'treno_arancio'
    elif re.search(r'treno viola', user_input, re.IGNORECASE):
        return 'treno_viola'
    return None

# Funzione di interfaccia del chatbot
def chatbot_interface():
    manual_chunks = {}
    memoria = []  # Inizializza la memoria della conversazione
    
    # Carica e divide i manuali
    for product, path in manuali_prodotti.items():
        manual_chunks[product] = carica_e_dividi_manuali(path)

    prodotto_corrente = None  # Variabile per tenere traccia del prodotto corrente

    print("Chatbot attivo: dimmi cosa vuoi fare. Digita 'esci' per terminare.")

    while True:
        user_input = input("Tu: ")  # Input dell'utente
        
        if user_input.lower() == "esci":
            break
        
        manual_name = seleziona_manuale(user_input)
        
        if manual_name:
            prodotto_corrente = manual_name  # Aggiorna il prodotto corrente
            risposta = genera_risposta(user_input, manual_chunks[manual_name], memoria)
            memoria.append(f"Utente: {user_input}\nChatbot: {risposta}")
            print(f"Tu: {user_input}")
            print(f"Chatbot: {risposta}")
        elif prodotto_corrente:
            risposta = genera_risposta(user_input, manual_chunks[prodotto_corrente], memoria)  # Usa il prodotto corrente
            memoria.append(f"Utente: {user_input}\nChatbot: {risposta}")
            print(f"Tu: {user_input}")
            print(f"Chatbot: {risposta}")
        else:
            print("Chatbot: Non ho trovato il prodotto specificato. Riprova.")

if __name__ == "__main__":
    chatbot_interface()


Chatbot attivo: dimmi cosa vuoi fare. Digita 'esci' per terminare.
Tu: devo assemblare la giraffa, che pezzi devo utilizzare?
Chatbot: Per assemblare la giraffa, hai bisogno dei seguenti pezzi:

- 2 pezzi Verde Grande
- 2 pezzi Celeste Triplo
- 4 pezzi Blu Unico
- 1 pezzo Arancione Punta

Se hai bisogno di ulteriori informazioni o istruzioni su una fase specifica, non esitare a chiedere.
Tu: quale è la prima fase?
Chatbot: La prima fase è l'Assemblaggio del Modulo Corpo. Segui le istruzioni per predisporre i 2 pezzi Verde Grande in modo parallelo e distanziati, quindi sovrapporre i 2 pezzi Celeste Triplo sui due agganci esterni dei pezzi Verde, unendo le estremità.
Tu: fase finale
Chatbot: La fase finale consiste nel fissare ogni Zampa agli agganci inferiori del corpo, installare il Collo e la Testa sull'aggancio superiore del corpo, orientando la testa verso l'esterno. Assicurati che tutti i componenti siano fissati correttamente e che non ci siano parti mancanti. Successivamente, pro